In [2]:
import pandas as pd
import glob

datapath = './collected/'
usfiles = glob.glob(datapath + 'us*.csv')
df = pd.read_csv(usfiles[0])
df

,id,conversation_id,referenced_tweets.replied_to.id,referenced_tweets.retweeted.id,referenced_tweets.quoted.id,author_id,in_reply_to_user_id,retweeted_user_id,quoted_user_id,created_at,...,geo.geo.bbox,geo.geo.type,geo.id,geo.name,geo.place_id,geo.place_type,__twarc.retrieved_at,__twarc.url,__twarc.version,Unnamed: 73
0,1410152713804029952,1410152713804029952,NaN,NaN,NaN,162218042,NaN,NaN,NaN,2021-06-30T08:26:20.000Z,...,"[-106.645646, 25.837092, -93.508131, 36.500695]",Feature,e0060cda70f5f341,Texas,e0060cda70f5f341,admin,2022-04-21T00:39:33+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.10.2,NaN
1,1410152710461136897,1409965094780407810,1.409965e+18,NaN,NaN,952415351106498560,4.546341e+09,NaN,NaN,2021-06-30T08:26:20.000Z,...,"[-86.339696, 43.201538, -86.144095, 43.264395]",Feature,012b416c2c776945,Muskegon,012b416c2c776945,city,2022-04-21T00:39:33+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.10.2,NaN
2,1410152696750002179,1409603818585735169,1.409944e+18,NaN,NaN,936561177819320320,1.475841e+09,NaN,NaN,2021-06-30T08:26:16.000Z,...,"[-87.940033, 41.644102, -87.523993, 42.0230669]",Feature,1d9a5370a355ab0c,Chicago,1d9a5370a355ab0c,city,2022-04-21T00:39:33+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.10.2,NaN
3,1410152691687432195,1410152691687432195,NaN,NaN,1.410077e+18,496542587,NaN,NaN,1.007085e+18,2021-06-30T08:26:15.000Z,...,"[-86.505805, 35.751433, -86.313415, 35.943407]",Feature,19e2bff2e89dc38e,Murfreesboro,19e2bff2e89dc38e,city,2022-04-21T00:39:33+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.10.2,NaN
4,1410152678798237697,1410147768329551873,1.410148e+18,NaN,NaN,31366271,2.215141e+08,NaN,NaN,2021-06-30T08:26:12.000Z,...,"[-88.097892, 37.771743, -84.78458, 41.761368]",Feature,1010ecfa7d3a40f8,Indiana,1010ecfa7d3a40f8,admin,2022-04-21T00:39:33+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.10.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1410152350308868101,1410152350308868101,NaN,NaN,NaN,2937943430,NaN,NaN,NaN,2021-06-30T08:24:54.000Z,...,"[-74.026675, 40.683935, -73.910408, 40.877483]",Feature,01a9a39529b27f36,Manhattan,01a9a39529b27f36,city,2022-04-21T00:39:33+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.10.2,NaN
96,1410152349688008705,1410152349688008705,NaN,NaN,NaN,1563271657,NaN,NaN,NaN,2021-06-30T08:24:54.000Z,...,"[-83.5218655, 33.877554, -83.277693, 34.035213]",Feature,01f9c9fd7bb1aa6b,Athens,01f9c9fd7bb1aa6b,city,2022-04-21T00:39:33+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.10.2,NaN
97,1410152345988599811,1410150393263190018,1.410150e+18,NaN,NaN,1237113976238071808,1.291527e+18,NaN,NaN,2021-06-30T08:24:53.000Z,...,"[-118.668404, 33.704538, -118.155409, 34.337041]",Feature,3b77caf94bfc81fe,Los Angeles,3b77caf94bfc81fe,city,2022-04-21T00:39:33+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.10.2,NaN
98,1410152344512204801,1409741661895286790,1.409742e+18,NaN,NaN,26972692,7.592510e+05,NaN,NaN,2021-06-30T08:24:52.000Z,...,"[-112.6265495, 33.355798, -112.461428, 33.5154...",Feature,0015cc0d71d49e19,Buckeye,0015cc0d71d49e19,city,2022-04-21T00:39:33+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.10.2,NaN


# descriptive data

In [7]:
jpfiles = glob.glob(datapath + 'jp*.csv')


In [5]:
len(usfiles)

8770

In [6]:
# count the total number of tweets for files starting with us
def count_tweets(files):
    count = 0
    for file in files:
        try:
            df = pd.read_csv(file)
            # delete duplicate id
            df = df.drop_duplicates(subset=['id'])
            count += len(df)
        except:
            print(file)
    return count

count_tweets(usfiles)

./collected/us_014509_01Jan2021.csv


1471856

In [8]:
count_tweets(jpfiles)

1300324

# calculate entropy by word count

In [1]:
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

def get_emotion_loadings(tweet_text):
    model_name = "cardiffnlp/twitter-roberta-base-emotion"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    emotion_classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
    result = emotion_classifier(tweet_text)

    # Extract emotion probabilities
    emotion_probabilities = [entry["score"] for entry in result]
    return emotion_probabilities

# Example usage
tweet_text = "I love this new phone! It's amazing."
emotion_loadings = get_emotion_loadings(tweet_text)
print("Emotion loadings:", emotion_loadings)

def emotion_entropy(post_emotions, threshold=0.5):
    binary_emotions = [1 if emotion >= threshold else 0 for emotion in post_emotions]
    probabilities = [emotion_count / len(post_emotions) for emotion_count in binary_emotions]
    
    entropy = -np.sum([p * np.log2(p) if p > 0 else 0 for p in probabilities])
    return entropy

# Example usage
post_emotions = [0.7, 0.2, 0.8, 0.4, 0.6, 0.1]
entropy = emotion_entropy(post_emotions)
print("Emotion entropy:", entropy)

tweet_text = "I love this new phone! It's amazing."
emotion_loadings = get_emotion_loadings(tweet_text)
entropy = emotion_entropy(emotion_loadings)
print("Emotion loadings:", emotion_loadings)
print("Emotion entropy:", entropy)


Emotion loadings: [0.9440009593963623]
Emotion entropy: 1.292481250360578
Emotion entropy: -0.0


## get dict

In [3]:
def load_en_nrc_emotion_lexicon(txt_path):
    with open(txt_path, 'r') as f:
        lines = f.readlines()
        
    data = [line.strip().split('\t') for line in lines]
    df = pd.DataFrame(data, columns=["word", "emotion", "value"])
    df["value"] = df["value"].astype(int)
    
    df = df[df["value"] == 1]
    emotion_dict = {emotion: set(df[df["emotion"] == emotion]["word"]) for emotion in df["emotion"].unique()}
    
    return emotion_dict

path_to_english_nrc = "./preprocessing/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"
english_nrc_emotion_dict = load_en_nrc_emotion_lexicon(path_to_english_nrc)


def load_jp_nrc_emotion_lexicon(file_path):
    emotion_dict = {"anger": set(), "anticipation": set(), "disgust": set(), "fear": set(), "joy": set(),
                    "negative": set(), "positive": set(), "sadness": set(), "surprise": set(), "trust": set()}

    with open(file_path, encoding="utf-8") as f:
        next(f)  # Skip the header line
        for line in f:
            split_line = line.strip().split("\t")
            japanese_word = split_line[-1]
            emotions = [emotion for emotion, value in zip(emotion_dict.keys(), split_line[1:-1]) if int(value) == 1]

            for emotion in emotions:
                emotion_dict[emotion].add(japanese_word.lower())

    return emotion_dict

path_to_japanese_nrc = "./preprocessing/Japanese-NRC-EmoLex.txt"
japanese_nrc_emotion_dict = load_jp_nrc_emotion_lexicon(path_to_japanese_nrc)


In [26]:
def get_emotion_loadings_english(tweet_text, eed):
    tokenized_tweet = word_tokenize(tweet_text)

    # Count the occurrences of words related to each emotion category
    basic_emotions = ['fear', 'anger', 'joy', 'sadness', 'disgust', 'surprise']
    emotion_counts = {emotion: 0 for emotion in eed if emotion in basic_emotions}
    for token in tokenized_tweet:
        for emotion, words in eed.items():
            if emotion in basic_emotions and token in words:
                emotion_counts[emotion] += 1

    # Calculate emotion loadings (normalize counts by the total number of emotion words)
    total_emotion_words = sum(emotion_counts.values())
    emotion_loadings = [count / total_emotion_words if total_emotion_words > 0 else 0 for count in emotion_counts.values()]

    return emotion_loadings

def get_emotion_loadings_japanese(tweet_text, jed):
    tokenized_tweet = list(jieba.cut(tweet_text))

    # Count the occurrences of words related to each emotion category
    basic_emotions = ['fear', 'anger', 'joy', 'sadness', 'disgust', 'surprise']
    emotion_counts = {emotion: 0 for emotion in jed if emotion in basic_emotions}
    for token in tokenized_tweet:
        for emotion, words in jed.items():
            if emotion in basic_emotions and token in words:
                emotion_counts[emotion] += 1

    # Calculate emotion loadings (normalize counts by the total number of emotion words)
    total_emotion_words = sum(emotion_counts.values())
    emotion_loadings = [count / total_emotion_words if total_emotion_words > 0 else 0 for count in emotion_counts.values()]

    return emotion_loadings


In [27]:
english_tweet_text = "This new phone is amazing! It's simply magical."  # Replace with your English tweet text
japanese_tweet_text = "この新しい携帯電話は素晴らしいです！ただただ魔法のようです。"  # Replace with your Japanese tweet text

english_emotion_loadings = get_emotion_loadings_english(english_tweet_text, english_nrc_emotion_dict)
japanese_emotion_loadings = get_emotion_loadings_japanese(japanese_tweet_text, japanese_nrc_emotion_dict)

english_entropy = emotion_entropy(english_emotion_loadings)
japanese_entropy = emotion_entropy(japanese_emotion_loadings)
print("English tweet emotion loadings:", english_emotion_loadings)
print("Japanese tweet emotion loadings:", japanese_emotion_loadings)
print("English tweet emotion entropy:", english_entropy)
print("Japanese tweet emotion entropy:", japanese_entropy)


English tweet emotion loadings: [0.0, 0.0, 0.0, 0.5, 0.0, 0.5]
Japanese tweet emotion loadings: [0.0, 0.5, 0.5, 0.0, 0.0, 0.0]
English tweet emotion entropy: 0.861654166907052
Japanese tweet emotion entropy: 0.861654166907052


In [28]:
mixed_emotion_sentences = [
    "I'm so happy to see you, but I'm sad that you have to leave soon.",
    "彼の成功は素晴らしいけれども、同時に少し嫉妬心がある",
    "I'm anxious about the exam, but I'm also excited about the challenge.",
    "仕事が終わったので安心だが、週末のパーティには緊張している",
    "I'm surprised and disappointed by the news.",
    "彼女の発表には驚いたが、ちょっと怒りも感じた"
]
english_emotion_loadings = [get_emotion_loadings_english(sentence, english_nrc_emotion_dict) for sentence in mixed_emotion_sentences[::2]]
japanese_emotion_loadings = [get_emotion_loadings_japanese(sentence, japanese_nrc_emotion_dict) for sentence in mixed_emotion_sentences[1::2]]
english_entropy = [emotion_entropy(loadings) for loadings in english_emotion_loadings]
japanese_entropy = [emotion_entropy(loadings) for loadings in japanese_emotion_loadings]
for i, sentence in enumerate(mixed_emotion_sentences):
    if i % 2 == 0:
        loadings = english_emotion_loadings[i // 2]
        entropy = english_entropy[i // 2]
    else:
        loadings = japanese_emotion_loadings[i // 2]
        entropy = japanese_entropy[i // 2]
    
    print(f"Sentence: {sentence}")
    print(f"Emotion Loadings: {loadings}")
    print(f"Entropy: {entropy:.2f}")
    print()


Sentence: I'm so happy to see you, but I'm sad that you have to leave soon.
Emotion Loadings: [0.0, 0.3333333333333333, 0.0, 0.3333333333333333, 0.0, 0.3333333333333333]
Entropy: 0.00

Sentence: 彼の成功は素晴らしいけれども、同時に少し嫉妬心がある
Emotion Loadings: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
Entropy: 0.43

Sentence: I'm anxious about the exam, but I'm also excited about the challenge.
Emotion Loadings: [0.4, 0.0, 0.2, 0.2, 0.0, 0.2]
Entropy: 0.00

Sentence: 仕事が終わったので安心だが、週末のパーティには緊張している
Emotion Loadings: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
Entropy: 0.43

Sentence: I'm surprised and disappointed by the news.
Emotion Loadings: [0.0, 0.25, 0.25, 0.25, 0.25, 0.0]
Entropy: 0.00

Sentence: 彼女の発表には驚いたが、ちょっと怒りも感じた
Emotion Loadings: [0, 0, 0, 0, 0, 0]
Entropy: 0.00



# entropy by DDR

In [ ]:
basic_emotions = ['anger', 'fear', 'joy', 'sadness', 'disgust', 'surprise']

japanese_basic_emotion_concepts = {
    emotion: list(words) for emotion, words in japanese_nrc_emotion_dict.items() if emotion in basic_emotions
}
english_basic_emotion_concepts = {
    emotion: list(words) for emotion, words in english_nrc_emotion_dict.items() if emotion in basic_emotions
}


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize

# Load pre-trained GloVe embeddings
def load_glove_embeddings(embeddings_file):
    embeddings = {}
    with open(embeddings_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.array(values[1:], dtype=np.float32)
            embeddings[word] = vector
    return embeddings

# Vectorize the dictionary of moral and emotional concepts
def vectorize_concepts(concepts, embeddings):
    concept_vectors = {}
    for concept, words in concepts.items():
        vectors = [embeddings[word] for word in words if word in embeddings]
        concept_vectors[concept] = np.mean(vectors, axis=0)
    return concept_vectors

# Vectorize the input text
def vectorize_text(text, embeddings):
    tokens = word_tokenize(text)
    vectors = [embeddings[token] for token in tokens if token in embeddings]
    return np.mean(vectors, axis=0)

# Calculate cosine similarity between input text and moral and emotional concepts
def calculate_similarities(text_vector, concept_vectors):
    similarities = {}
    for concept, vector in concept_vectors.items():
        similarities[concept] = cosine_similarity(text_vector.reshape(1, -1), vector.reshape(1, -1))[0][0]
    return similarities

# Example usage:
glove_embeddings_file = 'glove.6B.300d.txt'
glove_embeddings = load_glove_embeddings(glove_embeddings_file)

# Define your moral and emotional concepts dictionary here
concepts = english_basic_emotion_concepts

concept_vectors = vectorize_concepts(concepts, glove_embeddings)

text = "This is an example text to analyze moral and emotional content."
text_vector = vectorize_text(text, glove_embeddings)

similarities = calculate_similarities(text_vector, concept_vectors)
print(similarities)


In [2]:
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from janome.tokenizer import Tokenizer

# Load pre-trained FastText embeddings
def load_fasttext_embeddings(embeddings_file):
    embeddings = {}
    with open(embeddings_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.array(values[1:], dtype=np.float32)
            embeddings[word] = vector
    return embeddings

# Vectorize the dictionary of moral and emotional concepts
def vectorize_concepts(concepts, embeddings):
    concept_vectors = {}
    for concept, words in concepts.items():
        vectors = [embeddings[word] for word in words if word in embeddings]
        concept_vectors[concept] = np.mean(vectors, axis=0)
    return concept_vectors

# Tokenize the input text using Janome
def tokenize_japanese_text(text):
    tokenizer = Tokenizer()
    tokens = tokenizer.tokenize(text, wakati=True)
    return tokens

# Vectorize the input text
def vectorize_text(tokens, embeddings):
    vectors = [embeddings[token] for token in tokens if token in embeddings]
    return np.mean(vectors, axis=0)

# Calculate cosine similarity between input text and moral and emotional concepts
def calculate_similarities(text_vector, concept_vectors):
    similarities = {}
    for concept, vector in concept_vectors.items():
        similarities[concept] = cosine_similarity(text_vector.reshape(1, -1), vector.reshape(1, -1))[0][0]
    return similarities

# Example usage:
fasttext_embeddings_file = 'cc.ja.300.vec'
fasttext_embeddings = load_fasttext_embeddings(fasttext_embeddings_file)

# Define your moral and emotional concepts dictionary here
concepts = japanese_basic_emotion_concepts

concept_vectors = vectorize_concepts(concepts, fasttext_embeddings)

text = "これは、道徳的および感情的なコンテンツを分析するための例文です。"
tokens = tokenize_japanese_text(text)
text_vector = vectorize_text(tokens, fasttext_embeddings)

similarities = calculate_similarities(text_vector, concept_vectors)
print(similarities)


ModuleNotFoundError: No module named 'janome'

In [9]:
!conda install scikit-learn


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/local/PSYCH-ADS/xuqian_chen/anaconda3/envs/jupyter_env/envs/twitter

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    joblib-1.2.0               |     pyhd8ed1ab_0         205 KB  conda-forge
    libblas-3.9.0              |   12_linux64_mkl          12 KB  conda-forge
    libcblas-3.9.0             |   12_linux64_mkl          12 KB  conda-forge
    libgfortran-ng-12.2.0      |      h69a702a_19          22 KB  conda-forge
    libgfortran5-12.2.0        |      h337968e_19         1.8 MB  conda-forge
    liblapack-3.9.0            |   12_linux64_mkl          12 KB  conda-forge
    scikit-le